In [30]:
# Imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import time
import sys
import chromedriver_binary
import re
from collections import defaultdict
from glob import glob

In [31]:
def fmt_url(url_string, items_per_page, page_nbr):
    return url_string.format(items_per_page * (page_nbr - 1))

In [32]:
URL = 'https://www.airbnb.com/s/seattle/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&screen_size=large&hide_dates_and_guests_filters=false&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&search_type=pagination&source=structured_search_input_header&checkin=2019-12-22&checkout=2019-12-28&adults=1&room_types%5B%5D=Entire%20home%2Fapt&s_tag=BnVpdDt-&section_offset=5&items_offset={}&last_search_session_id=38f56942-96ed-430e-a723-bc7e1b7f9947'

In [33]:
driver = webdriver.Chrome()
# driver.get(url2)

In [34]:
list_of_links = []

In [35]:
window = driver.window_handles[0]

In [36]:
for i in range(1, 18):
    url2 = fmt_url(URL, 18, i)
    driver.switch_to.window(window)
    driver.get(url2)
    time.sleep(10)
    dict_of_urls = defaultdict(int)
    a = driver.find_elements_by_tag_name('a')
    for element in a:
        link = element.get_attribute('href')
        if not link is None and link.find('rooms') != -1:
            dict_of_urls[link] += 1
    list_of_links += list(dict_of_urls.keys())

In [37]:
print(len(list_of_links))
list_of_links[:5]

306


['https://www.airbnb.com/rooms/34127978?location=seattle&adults=1&check_in=2019-12-22&check_out=2019-12-28&previous_page_section_name=1000',
 'https://www.airbnb.com/rooms/22750242?location=seattle&adults=1&check_in=2019-12-22&check_out=2019-12-28&previous_page_section_name=1000',
 'https://www.airbnb.com/rooms/29736429?location=seattle&adults=1&check_in=2019-12-22&check_out=2019-12-28&previous_page_section_name=1000',
 'https://www.airbnb.com/rooms/32822782?location=seattle&adults=1&check_in=2019-12-22&check_out=2019-12-28&previous_page_section_name=1000',
 'https://www.airbnb.com/rooms/39066335?location=seattle&adults=1&check_in=2019-12-22&check_out=2019-12-28&previous_page_section_name=1000']

In [20]:
import pandas as pd

In [38]:
pd.DataFrame(pd.Series(list_of_links), columns=['links']).to_csv('links3.csv', index=False)

In [22]:
list_of_links[0] 

'https://www.airbnb.com/rooms/29736429?location=seattle&adults=1&check_in=2019-12-22&check_out=2019-12-28&previous_page_section_name=1000'

In [39]:
link_files = glob('/Users/scott/metis/projects/project2/notebooks/links*')
link_files

['/Users/scott/metis/projects/project2/notebooks/links.csv',
 '/Users/scott/metis/projects/project2/notebooks/links1.csv',
 '/Users/scott/metis/projects/project2/notebooks/links3.csv',
 '/Users/scott/metis/projects/project2/notebooks/links2.csv']

In [40]:
links_df = pd.concat([pd.read_csv(file) for file in link_files])

In [41]:
links_df.head()

,links
0,https://www.airbnb.com/rooms/29736429?location...
1,https://www.airbnb.com/rooms/22750242?location...
2,https://www.airbnb.com/rooms/32822782?location...
3,https://www.airbnb.com/rooms/34127978?location...
4,https://www.airbnb.com/rooms/30093335?location...


In [42]:
links_df.drop_duplicates(inplace=True)

In [43]:
links_df.shape

(962, 1)

In [47]:
type(links_df.iloc[0][0])

str

In [52]:
xpaths = {'guests' : '//*[@id="room"]/div[2]/div/div[2]/div[1]/div/div[3]/div/div/div/div[1]/div[2]/div[2]/div[1]/div',
          'bedrooms' : '//*[@id="room"]/div[2]/div/div[2]/div[1]/div/div[3]/div/div/div/div[1]/div[2]/div[2]/div[2]/div',
          'beds' : '//*[@id="room"]/div[2]/div/div[2]/div[1]/div/div[3]/div/div/div/div[1]/div[2]/div[2]/div[3]/div',
          'baths' : '//*[@id="room"]/div[2]/div/div[2]/div[1]/div/div[3]/div/div/div/div[1]/div[2]/div[2]/div[4]/div',
          'narrative' : '//*[@id="details"]/div/div[1]/div/div/div/p/span/span',
          'amenities' : '//*[@id="amenities"]/div/div/div/div/div/section/div[4]/div/button',
          'price' : '//*[@id="room"]/div[2]/div/div[2]/div[2]/div/div/div[1]/div/div/div[1]/div/div/div[2]/div[1]/div/span[2]/span',
          'rules' : '//*[@id="house-rules"]/div/section/div[3]/div[1]/div/div/div[2]/div[2]',
          'cleaning_fee' : '//*[@id="book_it_form"]/div[2]/div[2]/div[1]/div[2]/span/span',
          'map' : '//*[@id="neighborhood"]/div/div/div/div/div/section/div[2]/div[2]/div[2]/img'
          }

In [48]:
driver = webdriver.Chrome()

In [51]:
def get_property_info(driver, link):
    elements_dict = {}
    driver.switch_to.window(driver.window_handles[0])
    driver.get(link)
    time.sleep(10)
    for key, val in xpaths.items():
        elements_dict[key] = 
    print(driver.page_source[:1000])
    return 0
get_property_info(driver, links_df.iloc[0][0])

<html data-is-hyperloop="true" lang="en" dir="ltr" xmlns:fb="http://ogp.me/ns/fb#" class="js-focus-visible"><head><style type="text/css">.gm-control-active>img{box-sizing:content-box;display:none;left:50%;pointer-events:none;position:absolute;top:50%;transform:translate(-50%,-50%)}.gm-control-active>img:nth-child(1){display:block}.gm-control-active:hover>img:nth-child(1),.gm-control-active:active>img:nth-child(1){display:none}.gm-control-active:hover>img:nth-child(2),.gm-control-active:active>img:nth-child(3){display:block}
</style><link type="text/css" rel="stylesheet" href="https://fonts.googleapis.com/css?family=Roboto:300,400,500,700|Google+Sans"><style type="text/css">.gm-ui-hover-effect{opacity:.6}.gm-ui-hover-effect:hover{opacity:1}
</style><style type="text/css">.gm-style .gm-style-cc span,.gm-style .gm-style-cc a,.gm-style .gm-style-mtc div{font-size:10px;box-sizing:border-box}
</style><style type="text/css">@media print {  .gm-style .gmnoprint, .gmnoprint {    display:none  }

0

In [ ]:
for link in list_of_links[4:6]:
    prop_dict = {}
    driver.get(link)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, "lxml")
    break

In [ ]:
# soup.find_all('div')
# soup.find_all('div', attrs={'style': "margin-right: 24px;"})

In [ ]:
tag = soup.find_all('div', attrs={'class' : "_czm8crp"})[0]

In [ ]:
tag.get_text()

In [ ]:
# return_url = driver.current_url
# return_url

In [ ]:
# return_url = return_url.replace('adults=1', 'adults=2')
# return_url

In [ ]:
# driver.find_element_by_class_name('_1m76pmy').click()

In [ ]:
driver.find_element_by_xpath('//*[@id="site-content"]/div/div/div[5]/div/div[1]/div').text

In [ ]:
explore = driver.find_element_by_xpath('//*[@id="site-content"]/div/div/div[4]/div/div/div')
explore.find_elements_by_xpath("//div[contains(@itemprop, 'itemListElement')]")[0].find_elements_by_tag_name('meta')[2].get_attribute('content')

In [ ]:
list_of_urls = []

for i in range(4):
    time.sleep(10)
    print(driver.find_element_by_xpath('//*[@id="site-content"]/div/div/div[5]/div/div[1]/div').text)
    explore = driver.find_element_by_xpath('//*[@id="site-content"]/div/div/div[4]/div/div/div')
    props = explore.find_elements_by_xpath("//div[contains(@itemprop, 'itemListElement')]")
    print(explore)
    print(len(props))
    if i == 0:
        # This class only has one instance ont he first page
        driver.find_element_by_class_name('_1m76pmy').click()
    else:
        # If we're not on the first page then choose the second instance of this class
        driver.find_elements_by_class_name('_1m76pmy')[1].click()


In [ ]:
list_of_urls = []

for i in range(4):
#     print(driver.current_url)
    print(driver.find_element_by_xpath('//*[@id="site-content"]/div/div/div[5]/div/div[1]/div').text)
    if i == 0:
        list_of_divs = driver.find_elements_by_xpath("//div[contains(@itemprop, 'itemListElement')]")
    else:
        list_of_divs = driver.find_elements_by_xpath('//*[@id="FMP-target"]/div/div/div/div/div[1]/div/div')
    for prop in list_of_divs:
        list_of_urls.append(prop.find_elements_by_tag_name('meta')[2].get_attribute('content'))
        # print(prop.find_elements_by_tag_name('meta')[2].get_attribute('content'))
        # break
    print('Len of urls:', len(list_of_urls))
    if i == 0:
        # This class only has one instance ont he first page
        driver.find_element_by_class_name('_1m76pmy').click()
    else:
        # If we're not on the first page then choose the second instance of this class
        driver.find_elements_by_class_name('_1m76pmy')[1].click()
    time.sleep(10)


In [ ]:
list_of_divs = driver.find_element_by_xpath("//div[contains(@itemprop, 'itemListElement')]")
list_of_divs

In [ ]:
driver.find_elements_by_xpath('//*[@id="FMP-target"]/div/div/div/div/div[1]/div/div')[0].find_elements_by_tag_name('meta')[2].get_attribute('content')

In [ ]:
for i, v in enumerate(list_of_urls):
    print(i, v)

In [ ]:
print(len(list_of_metas))
for prop in list_of_metas:
    # list_of_metas[0].find_elements_by_tag_name('meta')[2].get_attribute('content')
    print(prop.find_elements_by_tag_name('meta')[2].get_attribute('content'))

In [ ]:
# for meta in list_of_metas:
#     print(meta.value_of_css_property('content'))

In [ ]:
# list_of_metas

In [ ]:
'//*[@id="FMP-target"]/div/div/div/div/div[1]/div/div/meta[3]'

In [ ]:
driver.find_element_by_class_name('_1ip5u88').

In [ ]:
'<a href="/s/seattle/homes?refinement_paths%5B%5D=%2Fhomes&amp;current_tab_id=home_tab&amp;selected_tab_id=home_tab&amp;screen_size=medium&amp;hide_dates_and_guests_filters=true&amp;place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&amp;s_tag=y6o2gwE0&amp;search_type=pagination&amp;section_offset=5&amp;items_offset=36" class="_1ip5u88" aria-busy="false"><div class="_1m76pmy"><svg viewBox="0 0 18 18" role="img" aria-label="Next" focusable="false" style="height: 0.75em; width: 0.75em; display: block; fill: currentcolor;"><path d="m4.29 1.71a1 1 0 1 1 1.42-1.41l8 8a1 1 0 0 1 0 1.41l-8 8a1 1 0 1 1 -1.42-1.41l7.29-7.29z" fill-rule="evenodd"></path></svg></div></a>'

In [ ]:
'//*[@id="FMP-target"]/div/div/div/div/div[1]/div/div/meta[3]'

In [ ]:
'https://www.airbnb.com/s/seattle/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&screen_size=large&hide_dates_and_guests_filters=true&place_id=ChIJVTPokywQkFQRmtVEaUZlJRA&s_tag=o6LsNXgV&search_type=pagination'

In [ ]:
while None in [None, None, None]:


In [ ]:
l = [1,1,1,None]
while l.remove(None):
    pass

In [ ]:
l

In [ ]:
def get_links(driver, url, page):
    url = url.format((page - 1) * 18)
    print(url)
    driver.get(url)
    time.sleep(4)
    dict_of_urls = defaultdict(int)
    a = driver.find_elements_by_tag_name('a')
    print(len(a))
    for element in a:
        link = element.get_attribute('href')
        if not link is None and link.find('rooms') != -1:
            dict_of_urls[link] += 1
    return list(dict_of_urls.keys())
get_links(driver, URL, 1)